# Problem Set 6
In this problem set you will implement SGD and SVRG and compare the two to each other, and also to GD.

## Upload Data Files
Run the cell below to upload `digits.zip` and `news.zip` from your local machine.

In [ ]:
from google.colab import files
print("Please upload digits.zip and news.zip:")
uploaded = files.upload()

## Imports and Data Loading Utilities

In [ ]:
import zipfile
import pandas as pd
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.io import mmread
from scipy import sparse
import time

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

---
# Problem 1: Stochastic Variance Reduced Gradient Descent (SVRG)

We compare GD, SGD, and SVRG for $\ell_2$-regularized logistic regression on the digits dataset.

**Objective:**
$$
F(\omega) = \frac{1}{n}\sum_{i=1}^{n} f_i(\omega) + \frac{\lambda}{2}\|\omega\|^2
$$
where $f_i(\omega) = -\left[y_i \log \sigma(\omega^\top x_i) + (1 - y_i)\log(1 - \sigma(\omega^\top x_i))\right]$ and $\sigma(z) = 1/(1+e^{-z})$.

In [ ]:
# Load digits data
def loaddata(filename):
    data = {}
    with zipfile.ZipFile(filename) as z:
        for fname in z.namelist():
            data[fname] = pd.read_csv(z.open(fname), sep=' ', header=None)
    return data

digits_dict = loaddata('./digits.zip')
print(digits_dict.keys())

X_train = digits_dict['X_digits_train.csv'].to_numpy(dtype=float)
X_test = digits_dict['X_digits_test.csv'].to_numpy(dtype=float)
y_train = digits_dict['y_digits_train.csv'].to_numpy(dtype=int).ravel()
y_test = digits_dict['y_digits_test.csv'].to_numpy(dtype=int).ravel()

print(f"Training set: X={X_train.shape}, y={y_train.shape}")
print(f"Test set:     X={X_test.shape}, y={y_test.shape}")
print(f"Classes: {np.unique(y_train)}")

In [ ]:
# Add bias column and normalize features
def preprocess(X_train, X_test):
    """Standardize features and add intercept."""
    mean = X_train.mean(axis=0)
    std = X_train.std(axis=0)
    std[std == 0] = 1.0  # avoid division by zero
    X_tr = (X_train - mean) / std
    X_te = (X_test - mean) / std
    # Add intercept
    X_tr = np.hstack([X_tr, np.ones((X_tr.shape[0], 1))])
    X_te = np.hstack([X_te, np.ones((X_te.shape[0], 1))])
    return X_tr, X_te

X_tr, X_te = preprocess(X_train, X_test)
n, d = X_tr.shape
print(f"After preprocessing: n={n}, d={d}")

In [ ]:
# ---- Core logistic regression functions ----

def sigmoid(z):
    """Numerically stable sigmoid."""
    return np.where(z >= 0,
                    1.0 / (1.0 + np.exp(-z)),
                    np.exp(z) / (1.0 + np.exp(z)))

def neg_log_likelihood(w, X, y, lam):
    """Compute negative log-likelihood with l2 regularization."""
    z = X @ w
    # Numerically stable: -y*log(sigma(z)) - (1-y)*log(1-sigma(z))
    # = -y*z + log(1+exp(z))  using the stable form
    nll = np.mean(np.where(z >= 0,
                           -y * z + np.log(1 + np.exp(-z)),
                           -(y) * z + z + np.log(1 + np.exp(-z))))
    # Simpler stable form: log(1+exp(z)) - y*z
    log1pexp = np.where(z >= 0, z + np.log(1 + np.exp(-z)), np.log(1 + np.exp(z)))
    nll = np.mean(log1pexp - y * z)
    reg = (lam / 2.0) * np.dot(w, w)
    return nll + reg

def full_gradient(w, X, y, lam):
    """Full gradient of the regularized NLL."""
    n = X.shape[0]
    p = sigmoid(X @ w)
    grad = (1.0 / n) * X.T @ (p - y) + lam * w
    return grad

def single_gradient(w, x_i, y_i, lam, n):
    """Gradient of f_i(w) + (lam/2)||w||^2.
    Note: The regularization is applied per sample to match the decomposition."""
    p = sigmoid(x_i @ w)
    grad = x_i * (p - y_i) + lam * w
    return grad

In [ ]:
# ---- Optimization Algorithms ----

def gradient_descent(X, y, lam, eta, num_passes, w_init=None):
    """Standard Gradient Descent.
    Each pass = 1 full gradient computation = n gradient evaluations.
    """
    n, d = X.shape
    w = np.zeros(d) if w_init is None else w_init.copy()
    
    nll_history = [neg_log_likelihood(w, X, y, lam)]
    grad_evals = [0]  # number of individual gradient evaluations
    
    for t in range(num_passes):
        g = full_gradient(w, X, y, lam)
        w = w - eta * g
        nll_history.append(neg_log_likelihood(w, X, y, lam))
        grad_evals.append((t + 1) * n)
    
    return w, nll_history, grad_evals


def sgd(X, y, lam, eta0, num_grad_evals, w_init=None, decay=True):
    """Stochastic Gradient Descent with 1/t learning rate decay.
    Records NLL every n gradient evaluations (1 effective pass).
    """
    n, d = X.shape
    w = np.zeros(d) if w_init is None else w_init.copy()
    
    nll_history = [neg_log_likelihood(w, X, y, lam)]
    grad_evals = [0]
    
    total_evals = 0
    t = 0
    while total_evals < num_grad_evals:
        i = np.random.randint(n)
        t += 1
        if decay:
            eta = eta0 / (1 + eta0 * lam * t)
        else:
            eta = eta0
        g_i = single_gradient(w, X[i], y[i], lam, n)
        w = w - eta * g_i
        total_evals += 1
        
        # Record every n evaluations (one effective pass)
        if total_evals % n == 0:
            nll_history.append(neg_log_likelihood(w, X, y, lam))
            grad_evals.append(total_evals)
    
    return w, nll_history, grad_evals


def svrg(X, y, lam, eta, S, T, w_init=None):
    """SVRG: Stochastic Variance Reduced Gradient.
    
    S = number of outer epochs (stages)
    T = number of inner loop iterations per stage
    
    Each outer epoch: 1 full gradient (n evals) + T inner updates (T evals)
    Total per epoch: n + T gradient evaluations.
    """
    n, d = X.shape
    w = np.zeros(d) if w_init is None else w_init.copy()
    
    nll_history = [neg_log_likelihood(w, X, y, lam)]
    grad_evals = [0]
    total_evals = 0
    
    for s in range(S):
        # Snapshot: compute full gradient at current w
        w_tilde = w.copy()
        mu = full_gradient(w_tilde, X, y, lam)
        total_evals += n  # full gradient costs n evaluations
        
        w_inner = w.copy()
        for t in range(T):
            i = np.random.randint(n)
            # Variance-reduced gradient estimate
            g_i_current = single_gradient(w_inner, X[i], y[i], lam, n)
            g_i_snapshot = single_gradient(w_tilde, X[i], y[i], lam, n)
            v_t = g_i_current - g_i_snapshot + mu
            w_inner = w_inner - eta * v_t
            total_evals += 2  # 2 individual gradient evaluations
        
        # Option I: set w to the last iterate of inner loop
        w = w_inner.copy()
        
        nll_history.append(neg_log_likelihood(w, X, y, lam))
        grad_evals.append(total_evals)
    
    return w, nll_history, grad_evals

In [ ]:
# ---- Choose l2 regularization parameter via test set performance ----

def test_accuracy(w, X, y):
    preds = (sigmoid(X @ w) >= 0.5).astype(int)
    return np.mean(preds == y)

best_lam = None
best_acc = 0
for lam_candidate in [1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1.0]:
    w_gd, _, _ = gradient_descent(X_tr, y_train, lam_candidate, eta=1.0, num_passes=100)
    acc = test_accuracy(w_gd, X_te, y_test)
    print(f"lambda={lam_candidate:.1e}, test accuracy={acc:.4f}")
    if acc > best_acc:
        best_acc = acc
        best_lam = lam_candidate

print(f"\nBest lambda = {best_lam}, test accuracy = {best_acc:.4f}")

In [ ]:
# ---- Run GD, SGD, SVRG with chosen lambda ----

lam = best_lam
num_passes_gd = 50  # 50 full passes for GD
total_grad_evals_budget = num_passes_gd * n  # same budget for all

# GD
eta_gd = 1.0
w_gd, nll_gd, evals_gd = gradient_descent(X_tr, y_train, lam, eta_gd, num_passes_gd)

# SGD with decaying step size
eta0_sgd = 1.0
w_sgd, nll_sgd, evals_sgd = sgd(X_tr, y_train, lam, eta0_sgd, total_grad_evals_budget)

# SVRG with T = 2n (standard choice)
T_svrg = 2 * n
# Number of outer stages to match budget: each stage costs n + 2T evals
cost_per_stage = n + 2 * T_svrg
S_svrg = total_grad_evals_budget // cost_per_stage
eta_svrg = 0.1  # typically smaller step size for SVRG
w_svrg, nll_svrg, evals_svrg = svrg(X_tr, y_train, lam, eta_svrg, S_svrg, T_svrg)

print(f"GD final NLL:   {nll_gd[-1]:.6f}")
print(f"SGD final NLL:  {nll_sgd[-1]:.6f}")
print(f"SVRG final NLL: {nll_svrg[-1]:.6f}")

In [ ]:
# ---- Plot: GD vs SGD vs SVRG ----

plt.figure(figsize=(10, 6))
plt.plot(np.array(evals_gd) / n, nll_gd, 'b-o', label='GD', markevery=5, markersize=6)
plt.plot(np.array(evals_sgd) / n, nll_sgd, 'r-s', label='SGD', markevery=5, markersize=5)
plt.plot(np.array(evals_svrg) / n, nll_svrg, 'g-^', label=f'SVRG (T=2n={T_svrg})', markevery=2, markersize=6)
plt.xlabel('Number of Effective Passes (gradient evaluations / n)', fontsize=13)
plt.ylabel('Negative Log-Likelihood (Training)', fontsize=13)
plt.title(f'GD vs SGD vs SVRG — Logistic Regression on Digits ($\\lambda$={lam})', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# ---- Effect of T on SVRG performance ----

plt.figure(figsize=(10, 6))

T_values = [n // 2, n, 2 * n, 5 * n]
colors = ['green', 'purple', 'orange', 'brown']
markers = ['^', 'D', 'v', 'p']

for T_val, color, marker in zip(T_values, colors, markers):
    cost = n + 2 * T_val
    S_val = total_grad_evals_budget // cost
    _, nll_vals, evals_vals = svrg(X_tr, y_train, lam, eta_svrg, S_val, T_val)
    plt.plot(np.array(evals_vals) / n, nll_vals, color=color, marker=marker,
             label=f'SVRG T={T_val} ({T_val/n:.1f}n)', markevery=max(1, len(nll_vals)//15), markersize=6)

# Also plot GD for reference
plt.plot(np.array(evals_gd) / n, nll_gd, 'b-o', label='GD', markevery=5, markersize=5, alpha=0.5)

plt.xlabel('Number of Effective Passes (gradient evaluations / n)', fontsize=13)
plt.ylabel('Negative Log-Likelihood (Training)', fontsize=13)
plt.title(f'Effect of T on SVRG Performance ($\\lambda$={lam})', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Discussion — Problem 1

**GD** converges steadily but each iteration is expensive (n gradient evaluations per step). 

**SGD** makes rapid initial progress since each iteration is cheap (1 gradient evaluation), but the inherent variance in the stochastic gradient estimates prevents it from achieving the same precision as GD — the loss oscillates and converges more slowly in later stages.

**SVRG** combines the best of both: it uses variance-reduced stochastic updates that converge at a linear rate (like GD) while keeping per-iteration cost low (like SGD). The full gradient snapshot periodically corrects the stochastic gradient, reducing variance.

**Effect of T (inner loop length):** 
- Too small T (e.g., T = n/2): The full gradient is recomputed too frequently relative to the inner work, making the overhead high.
- T ≈ 2n is a standard choice that balances the cost of the snapshot gradient with enough inner iterations to make progress.
- Too large T (e.g., T = 5n): The snapshot gradient becomes stale, which can slow convergence.

---
# Problem 2: Newsgroup Dataset Optimization

We optimize logistic regression on the high-dimensional newsgroup dataset using an advanced method (SGD with momentum + adaptive learning rate via Adam) and compare against standard SGD.

**Key strategies:**
1. **Sparse matrix operations** — the data is stored in sparse format and we avoid converting to dense.
2. **Mini-batch SGD** — improves gradient estimates without full passes.
3. **Adam optimizer** — adaptive per-parameter learning rates handle the high-dimensional sparse features effectively.
4. **L2 regularization** tuned for test performance.

In [ ]:
# Load news data
def loadnewsdata(filename='./news.zip'):
    data = {}
    with zipfile.ZipFile(filename) as z:
        for fname in z.namelist():
            if 'csv' in fname:
                data[fname] = pd.read_csv(z.open(fname), sep=' ', header=None)
            elif 'mtx' in fname:
                data[fname] = mmread(z.open(fname))
            else:
                raise Exception('unexpected filetype')
    return data

news_dict = loadnewsdata('./news.zip')
print(news_dict.keys())

X_news_train = news_dict['X_news_train.mtx']
X_news_test = news_dict['X_news_test.mtx']
y_news_train = news_dict['y_news_train.csv'].to_numpy(dtype=int).ravel()
y_news_test = news_dict['y_news_test.csv'].to_numpy(dtype=int).ravel()

# Convert to CSR for efficient row slicing
X_news_train = sparse.csr_matrix(X_news_train)
X_news_test = sparse.csr_matrix(X_news_test)

n_news, d_news = X_news_train.shape
print(f"Training: n={n_news}, d={d_news}")
print(f"Test:     n={X_news_test.shape[0]}")
print(f"Sparsity: {X_news_train.nnz / (n_news * d_news) * 100:.2f}% non-zero")
print(f"Classes: {np.unique(y_news_train)}")

In [ ]:
# ---- Sparse-aware logistic regression functions ----

def sigmoid_safe(z):
    """Numerically stable sigmoid for arrays."""
    return np.where(z >= 0,
                    1.0 / (1.0 + np.exp(-z)),
                    np.exp(z) / (1.0 + np.exp(z)))

def nll_sparse(w, X, y, lam):
    """NLL for sparse X."""
    z = np.array(X @ w).ravel()
    log1pexp = np.where(z >= 0, z + np.log(1 + np.exp(-z)), np.log(1 + np.exp(z)))
    nll = np.mean(log1pexp - y * z)
    return nll + (lam / 2.0) * np.dot(w, w)

def grad_sparse(w, X, y, lam):
    """Full gradient for sparse X."""
    n = X.shape[0]
    z = np.array(X @ w).ravel()
    p = sigmoid_safe(z)
    residual = p - y
    grad = np.array(X.T @ residual).ravel() / n + lam * w
    return grad

def minibatch_grad_sparse(w, X, y, indices, lam):
    """Mini-batch gradient for sparse X."""
    X_batch = X[indices]
    y_batch = y[indices]
    mb = len(indices)
    z = np.array(X_batch @ w).ravel()
    p = sigmoid_safe(z)
    residual = p - y_batch
    grad = np.array(X_batch.T @ residual).ravel() / mb + lam * w
    return grad

In [ ]:
# ---- Standard SGD for news dataset (baseline) ----

def sgd_sparse(X, y, lam, eta0, num_grad_evals, batch_size=1):
    """Standard SGD with decaying learning rate on sparse data."""
    n, d = X.shape
    w = np.zeros(d)
    
    nll_history = [nll_sparse(w, X, y, lam)]
    grad_evals = [0]
    total_evals = 0
    t = 0
    
    while total_evals < num_grad_evals:
        indices = np.random.randint(0, n, size=batch_size)
        t += 1
        eta = eta0 / (1 + eta0 * lam * t)
        g = minibatch_grad_sparse(w, X, y, indices, lam)
        w = w - eta * g
        total_evals += batch_size
        
        if total_evals % n == 0 or total_evals >= num_grad_evals:
            nll_history.append(nll_sparse(w, X, y, lam))
            grad_evals.append(total_evals)
    
    return w, nll_history, grad_evals


# ---- Adam Optimizer (our advanced method) ----

def adam_sparse(X, y, lam, eta=0.001, beta1=0.9, beta2=0.999, eps=1e-8,
                num_grad_evals=None, batch_size=64):
    """Adam optimizer with mini-batches on sparse data."""
    n, d = X.shape
    w = np.zeros(d)
    m = np.zeros(d)  # first moment
    v = np.zeros(d)  # second moment
    
    nll_history = [nll_sparse(w, X, y, lam)]
    grad_evals = [0]
    total_evals = 0
    t = 0
    
    while total_evals < num_grad_evals:
        indices = np.random.randint(0, n, size=batch_size)
        g = minibatch_grad_sparse(w, X, y, indices, lam)
        t += 1
        total_evals += batch_size
        
        # Adam updates
        m = beta1 * m + (1 - beta1) * g
        v = beta2 * v + (1 - beta2) * g**2
        m_hat = m / (1 - beta1**t)
        v_hat = v / (1 - beta2**t)
        w = w - eta * m_hat / (np.sqrt(v_hat) + eps)
        
        if total_evals % n == 0 or total_evals >= num_grad_evals:
            nll_history.append(nll_sparse(w, X, y, lam))
            grad_evals.append(total_evals)
    
    return w, nll_history, grad_evals

In [ ]:
# ---- Tune lambda on test set ----

def test_acc_sparse(w, X, y):
    z = np.array(X @ w).ravel()
    preds = (sigmoid_safe(z) >= 0.5).astype(int)
    return np.mean(preds == y)

print("Tuning lambda using Adam (quick runs)...")
best_lam_news = None
best_acc_news = 0
quick_budget = 20 * n_news  # 20 passes

for lam_c in [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]:
    w_temp, _, _ = adam_sparse(X_news_train, y_news_train, lam_c,
                               eta=0.001, num_grad_evals=quick_budget, batch_size=64)
    acc = test_acc_sparse(w_temp, X_news_test, y_news_test)
    print(f"  lambda={lam_c:.1e}, test accuracy={acc:.4f}")
    if acc > best_acc_news:
        best_acc_news = acc
        best_lam_news = lam_c

print(f"\nBest lambda = {best_lam_news}, test accuracy = {best_acc_news:.4f}")

In [ ]:
# ---- Run Standard SGD vs Adam ----

lam_news = best_lam_news
budget = 50 * n_news  # 50 effective passes

print("Running Standard SGD...")
t0 = time.time()
w_sgd_news, nll_sgd_news, evals_sgd_news = sgd_sparse(
    X_news_train, y_news_train, lam_news, eta0=0.1,
    num_grad_evals=budget, batch_size=1)
print(f"  SGD done in {time.time()-t0:.1f}s, final NLL={nll_sgd_news[-1]:.6f}")

print("Running Adam (mini-batch=64)...")
t0 = time.time()
w_adam_news, nll_adam_news, evals_adam_news = adam_sparse(
    X_news_train, y_news_train, lam_news, eta=0.001,
    num_grad_evals=budget, batch_size=64)
print(f"  Adam done in {time.time()-t0:.1f}s, final NLL={nll_adam_news[-1]:.6f}")

# Test accuracies
print(f"\nTest accuracy — SGD:  {test_acc_sparse(w_sgd_news, X_news_test, y_news_test):.4f}")
print(f"Test accuracy — Adam: {test_acc_sparse(w_adam_news, X_news_test, y_news_test):.4f}")

In [ ]:
# ---- Plot: Standard SGD vs Adam on News dataset ----

plt.figure(figsize=(10, 6))
plt.plot(np.array(evals_sgd_news) / n_news, nll_sgd_news, 'r-s',
         label='Standard SGD', markevery=max(1, len(nll_sgd_news)//15), markersize=5)
plt.plot(np.array(evals_adam_news) / n_news, nll_adam_news, 'b-o',
         label='Adam (mini-batch=64)', markevery=max(1, len(nll_adam_news)//15), markersize=5)
plt.xlabel('Number of Effective Passes (gradient evaluations / n)', fontsize=13)
plt.ylabel('Negative Log-Likelihood (Training)', fontsize=13)
plt.title(f'Standard SGD vs Adam — Newsgroup Dataset ($\\lambda$={lam_news})', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Discussion — Problem 2

**Methodology:**

1. **Sparse operations:** The newsgroup feature matrix is very high-dimensional and sparse. All matrix operations use scipy sparse CSR format, which avoids multiplying zeros and keeps memory usage manageable.

2. **Mini-batch SGD with Adam:** We use the Adam optimizer with mini-batches of size 64. Adam maintains per-parameter adaptive learning rates via exponential moving averages of the gradient and its square. This is especially beneficial for sparse, high-dimensional data because:
   - Features that appear rarely get effectively larger learning rates, allowing the model to learn from infrequent but informative words.
   - The momentum component smooths out noisy gradient estimates.
   - Mini-batches reduce gradient variance compared to single-sample SGD while still being much cheaper than full gradient computation.

3. **L2 regularization** was tuned on the test set to prevent overfitting in the high-dimensional setting.

**Results:** Adam with mini-batches converges significantly faster than standard SGD in terms of gradient evaluations. The adaptive learning rates and momentum allow Adam to navigate the loss landscape efficiently despite the high dimensionality and sparsity of the data. Standard SGD struggles because a single global learning rate cannot simultaneously serve the many features that vary in frequency and informativeness.